In [1]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
import os, sys
sys.path.append('..')
import common, sql
import pandas as pd
import numpy as np
hm={
    'Maven Security Versions': 'MSV',
    'Github Dependabot' : 'Dependabot',
    'OWASP Dependency-Check' : 'OWASP DC',
    'Contrast' : 'Com. B',
    'SourceClear': 'Com. A'
}

In [ ]:
q = '''select *
from mavenAlert mA
join dependency d on mA.dependencyId = d.id
join derivedMavenDependencyTree dMDT on d.id = dMDT.dependencyId
join tool t on mA.toolId = t.id;'''
df = pd.DataFrame(sql.execute(q))
gb = df.groupby('name')
for name, group in gb:
    repos = group.groupby('repositoryId')
    dRate = []
    d =t = 0
    for id, repo in repos:
        direct = len(repo[repo['depth'] == 1])
        transitive = len(repo[repo['depth'] > 1])
        total = direct + transitive
        d+= direct
        t+= transitive
        if total > 0:
            rate = round(direct/total * 100,1)
            dRate.append(rate)
    print(name, np.median(np.array(dRate)), round((d/(d+t))*100,1))

In [2]:
q = '''select *
from npmAlert nA
join dependency d on nA.dependencyId = d.id
left join
(select repositoryId,packageId, min(depth) as depth
from npmDependencyTree nDT
group by repositoryId,packageId) as dir
on dir.repositoryId=d.repositoryId and dir.packageId=d.packageId
join tool t on nA.toolId = t.id;'''
df = pd.DataFrame(sql.execute(q))
gb = df.groupby('name')
for name, group in gb:
    repos = group.groupby('repositoryId')
    dRate = []
    d =t = 0
    for id, repo in repos:
        direct = len(repo[repo['depth'] == 1])
        transitive = len(repo[repo['depth'] > 1])
        total = direct + transitive
        d+= direct
        t+= transitive
        if total > 0:
            rate = round(direct/total * 100,1)
            dRate.append(rate)
    print(name, np.median(np.array(dRate)), round((d/(d+t))*100,1))

Github Dependabot 100.0 65.9
NPM Audit 0.0 0.9
OWASP Dependency-Check 3.4000000000000004 4.4
Snyk 3.15 3.5
WhiteSource 10.75 8.8
